In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_transaction_fc")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.POS_WCM_SALEOUT_TRANLINE, VCM_STG_INF.POS_WCM_SALEOUT_PROMOTION"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc
(
  hash_id BIGINT,
  calday DATE,
  tran_no STRING,
  receipt_no STRING,
  product_id STRING,
  uom STRING,
  pos_quantity DOUBLE,
  amount DOUBLE,
  store_id STRING,
  promo_quantity DOUBLE,
  promo_amount DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_f_transaction_fc AS
    WITH full_vw AS (
        SELECT
            main_tbl.calday,
            main_tbl.tran_no,
            main_tbl.receipt_no,
            main_tbl.product_id,
            main_tbl.uom,
            main_tbl.sale_qty pos_quantity,
            main_tbl.sale_amt amount,
            main_tbl.store_id,
            SUM(item_qty) promo_quantity,
            SUM(discount_amount) promo_amount
        FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_tranline main_tbl
        LEFT JOIN {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_promotion discount
        USING (receipt_no, tran_no)
        WHERE main_tbl.store_id IN (
            SELECT store_id
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
            WHERE business_unit = '6000'
        )
        GROUP BY 1,2,3,4,5,6,7,8
    )
    SELECT
        farm_fingerprint(CONCAT(
            IFNULL(CAST(receipt_no AS STRING), ''),
            IFNULL(CAST(tran_no AS STRING), '')
        )) AS hash_id,
        *
    FROM full_vw WHERE CALDAY IN ({CALDAY_IN_STR})
    """)

else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")


In [0]:
# Get the calday list string from temp table
calday_query = f"""
SELECT
    CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
FROM temp_f_transaction_fc
"""

calday_df = spark.sql(calday_query)
calday_in_str = calday_df.collect()[0][0]

# Only run DELETE if the string is non-empty
if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
    spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc
    WHERE calday IN ({calday_in_str})
    """)
else:
    print("No valid CALDAYs found to delete.")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_fc can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_f_transaction_fc""")